In [1]:
from tournament import tournament
from reporting import Reporting
import importlib
from collections import namedtuple
from game_agent_comp import CustomPlayerComp
from value_functions import improved_score_fast_x2,improved_score_fast,\
    improved_score_fast_x3, partition_score_x2
from sample_players import null_score
from policy import SimplePolicy
import numpy as np
from copy import copy
#importlib.reload(reporting)

In [21]:
with open('data/calibrated_0_epoch_1.pickle', 'rb') as handle:
    test= pickle.load(handle)
test['function'].nn.coeff


array([ 0.        ,  0.        ,  0.        , ...,  0.06358748,
        0.0624285 ,  0.01280586])

In [2]:
import pickle, glob

files = glob.glob('data/result*.pickle')
print(files)

depths = {}
for file in files:#[:3]:
    try:
        with open(file, 'rb') as handle:
            old_depths = pickle.load(handle)
            #print(old_depths)
    except:
        old_depths={}

    #print(old_depths)
    for d in old_depths:
        for p, v in d.items():
            if p not in depths:
                depths[p]=v
            else:
                depths[p] = depths[p]+v

#with open('result.pickle', 'wb') as handle:
#    pickle.dump(depths, handle)
print(depths.keys())



['data\\result0.pickle', 'data\\result1.pickle', 'data\\result2.pickle']
dict_keys(['Faster improved', 'improved, two steps exact, with reporting', 'improved, three steps exact'])


In [3]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

example = depths['improved, three steps exact'][0]
move = example[1]
print(move['allscores'])

from value_functions import to_index

def nice_allscores(x):
    scores =[score for score, cell in x]
    inds = [to_index(cell) for score, cell in x]
    return softmax(scores), inds
    
print(nice_allscores(move['allscores']))

[(-3.0, (0, 5)), (1.0, (1, 4)), (1.0, (3, 4))]
(array([ 0.00907471,  0.49546264,  0.49546264]), [35, 29, 31])


In [4]:
# collate all moves in one big list
moves = []
final_moves =[]
#for player, games in depths.items():
games=depths['improved, three steps exact']
for game in games:
    for m, move in enumerate(game):
        if move['score'] != float('inf') and move['score'] != float('-inf'):
            moves.append(move)
        else:
            move_ = copy(move)
            if move['score'] == float('inf'):
                move_['score'] = 1
            else:
                move_['score'] = -1
            final_moves.append(move_)
            
depths = {}            
        
print(len(moves), len(final_moves))
print(nice_allscores(moves[0]['allscores']))

211158 44303
(array([ 0.11357902,  0.11357902,  0.04178339,  0.30873978,  0.11357902,
        0.30873978]), [34, 30, 24, 44, 48, 26])


In [5]:
np.zeros([0,5])

array([], shape=(0, 5), dtype=float64)

In [15]:


from neural import NNValueFunction, SelectionValueFunction, SingleValueFunction
import numpy as np
from copy import copy
from sklearn.model_selection import train_test_split
import math 
import random

train_final_scores = True

s = 1
if train_final_scores:
    train_moves, test_moves = train_test_split(final_moves, test_size = 0.1)
    sizes = [[5], [2, 2, 2], [5,5], [8,8]]
    val = SingleValueFunction(sizes[s])
else:
    train_moves, test_moves = train_test_split(moves, test_size = 0.1)
    sizes = [[5], [ 2, 2, 2] ,[5,5,5,5,5], [8,8,8,8] ]
    val = SelectionValueFunction(sizes[s])

coeff = np.random.normal(size =val.coeff_len)*0.1
n = 0
while 53*n < len(coeff):
    coeff[53*n:(53*n+10)] = 0
    n+=1
    
val.set_coeff(coeff)



n = 0
train_diff = np.zeros(len(train_moves))
train_base = copy(train_diff)
test_diff = np.zeros(len(test_moves))
test_base = copy(test_diff)

timed = False
while True:
    print('entering calibration next epoch...')
    dcoeff = np.zeros(val.coeff_len)
    random.shuffle(train_moves)
    # just run one batch
    for m, move in  enumerate(train_moves[:100]):
        if train_final_scores:
            thisvalue = val(input_vec = move['game'],pos = move['pos'], mask = move['game'])
            diff = move['score'] - thisvalue
            train_base[m] = move['score']
        else:
            scores, inds = nice_allscores(move['allscores'])
            thisvalue = val(input_vec = move['game'],pos = move['pos'], indices = inds, mask = move['game'])
            diff = scores - thisvalue
            train_base[m] = np.linalg.norm(scores - scores.mean())
        train_diff[m] = np.linalg.norm(diff)
        
        #square_print(nn(inp))
        #print(np.linalg.norm(diff))
        gr = val.nn.grad()
        #gr = gr /(1 + np.linalg.norm(gr))
        delta = gr.dot(diff).transpose()*0.01
        #print(diff,dcoeff,delta)
        try:
            dcoeff += delta
        except:
            dcoeff += delta[0]
            
        if m%20 ==0:
            #print(np.linalg.norm(dcoeff), np.linalg.norm(val.nn.get_coeff()))
            #dnorm = np.linalg.norm(dcoeff)
            #cnorm = np.linalg.norm(val.nn.get_coeff())
            #dcoeff = math.sqrt(cnorm/dnorm)*dcoeff
            #if m%50 == 0 and m>0:
                #print(np.linalg.norm(dcoeff), np.linalg.norm(val.nn.get_coeff()))
                #print('test: ',np.linalg.norm(train_diff)/np.linalg.norm(train_base))
            coeff = val.nn.get_coeff() + dcoeff
            #print(coeff.shape)
            val.nn.set_coeff(coeff)
            dcoeff = np.zeros(val.coeff_len)
        #if m%100 == 0
    random.shuffle(test_moves)
    for m,move in enumerate(test_moves[:500]):
        if train_final_scores:
            thisvalue = val(input_vec = move['game'],pos = move['pos'], mask = move['game'])
            diff = move['score'] - thisvalue
            test_base[m] = move['score']
        else:
            scores, inds = nice_allscores(move['allscores'])
            thisvalue = val(input_vec = move['game'],pos = move['pos'], indices = inds, mask = move['game'])
            diff = scores - thisvalue
            test_base[m] = np.linalg.norm(scores-scores.mean())
            
        test_diff[m] = np.linalg.norm(diff)
        
    print(np.linalg.norm(dcoeff), np.linalg.norm(val.nn.get_coeff()))
    print('train: ',(np.linalg.norm(train_diff)/np.linalg.norm(train_base))**2)
    print('test: ',(np.linalg.norm(test_diff)/np.linalg.norm(test_base))**2)


entering calibration next epoch...
0.0709542857667 2.6245149979
train:  0.956618290864
test:  0.901116651644
entering calibration next epoch...
0.0558134872366 2.64346403814
train:  0.884636683535
test:  0.744015548106
entering calibration next epoch...
0.0367066977184 2.67451512226
train:  0.733540846196
test:  0.600661669008
entering calibration next epoch...
0.0246428190834 2.70981546783
train:  0.473999578046
test:  0.573272089615
entering calibration next epoch...
0.0166163677248 2.70838519744
train:  0.338061189731
test:  0.62099336791
entering calibration next epoch...
0.0206886409297 2.69137343948
train:  0.520054599811
test:  0.530711807392
entering calibration next epoch...
0.0755677949365 2.72340060095
train:  0.404134705072
test:  0.614946978773
entering calibration next epoch...


KeyboardInterrupt: 

In [ ]:
from neural import NNValueFunction
import numpy as np
from copy import copy
from sklearn.model_selection import train_test_split

train_moves, test_moves = train_test_split(moves, test_size = 0.2)


val = NNValueFunction([1,3,3])
val.set_coeff(np.random.normal(size =val.coeff_len)*0.3)
n = 0
train_diff = np.zeros(len(train_moves))
train_base = copy(train_diff)
test_diff = np.zeros(len(test_moves))
test_base = copy(test_diff)
while True:
    for m, move in  enumerate(train_moves):
        train_diff[m] = move['score'] - val(input_vec = move['game'], pos = move['pos'])
        train_base[m] = move['score']
        #square_print(nn(inp))
        #print(np.linalg.norm(diff))
        gr = val.nn.grad()
        gr = gr /(1 + np.linalg.norm(gr))
        coeff = val.nn.get_coeff() + gr.dot(train_diff[m]).transpose()[0]*0.001
        #print(coeff.shape)
        val.nn.set_coeff(coeff)
        #if m%100 == 0
    for m,move in enumerate(test_moves):
        test_diff[m] = move['score'] - val(input_vec = move['game'], pos = move['pos'])
        test_base[m] = move['score']
    print('train: ',np.linalg.norm(train_diff)/np.linalg.norm(train_base))
    print('test: ',np.linalg.norm(test_diff)/np.linalg.norm(test_base))


In [ ]:
def transpose_list_of_lists(lol):
    
    transp = []

    for game in lol:
        for m, move_depth in enumerate(game):
            if len(transp) <= m:
                transp.append([])
            transp[m].append(move_depth)
    return transp



In [ ]:
#%pdb
from statistics import mean, stdev, median
import numpy as np

def get_mean_std(lol):
    result = {'mu':[], 'std':[],'median':[], 'strict_win':[], 'strict_loss':[]}
    
    for move in lol:
        if len(move) > 2:
            result['mu'].append(mean([m['depth'] for m in move]))
            result['std'].append(stdev([m['depth'] for m in move]))
            result['median'].append(median([m['depth'] for m in move]))
            result['strict_win'].append(\
                len([ m['score'] for m in move if m['score'] == float('inf')]) / len(move))
            result['strict_loss'].append(\
                len([ m['score'] for m in move if m['score'] == float('-inf')]) / len(move))
            
            
    result['mu'] = np.array(result['mu'] )
    result['std'] = np.array(result['std'])
    result['median'] = np.array(result['median'])
    result['strict_win'] = np.array(result['strict_win'])
    result['strict_loss'] = np.array(result['strict_loss'])
    return result

player_stats = {}
for p, d in depths.items():
    tmp = transpose_list_of_lists(d)
    #print(tmp)
    player_stats[p] = get_mean_std(transpose_list_of_lists(d))


In [ ]:
player_stats


In [ ]:
#import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
ax = plt.axes()
#ax.set_yscale("log")
lines = []
legend = []
n=0
lastpt = 12
for p, stats in player_stats.items():
    lines.append(\
        plt.errorbar(np.array(range(lastpt))+n,stats['mu'][0:lastpt], 
                     yerr=stats['std'][0:lastpt], fmt='o'))
    legend.append(p)
    n+= 0.1

plt.legend(lines,legend)
plt.show()

In [ ]:
plt.figure()
ax = plt.axes()
#ax.set_yscale("log")
lines = []
legend =[]
for p, stats in player_stats.items():
    lines.append(\
        plt.plot(range(len(stats['mu'])),stats['strict_win'] + stats['strict_loss']))
    legend.append(p)
plt.legend(legend)
plt.grid()
plt.show()

In [ ]:
plt.figure()
ax = plt.axes()
#ax.set_yscale("log")
lines = []
legend =[]
for p, stats in player_stats.items():
    lines.append(\
        plt.plot(range(len(stats['mu'])),stats['strict_win']-stats['strict_loss']))
    legend.append(p)
plt.legend(legend)
plt.grid()
plt.show()